In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from src.databases import Postgresql

# Criando lista de skus e urls

In [2]:
#Sku
lst_sku=[]
for i in range(1,84):
    PAGE = f'https://www.wine.com.br/vinhos/cVINHOS-p{i}.html'
    if i % 10 == 0:
        print(f'Raspando pagina {i}')

    resultado = requests.get(PAGE)
    bs = BeautifulSoup(resultado.content, 'html.parser')
    html_data = bs.find_all('div', {'class': 'ProductDisplay-name'})
    sku = np.array([elem.a['data-product-sku'] for elem in html_data])
    
    for elem in html_data:
        lst_sku.append(elem.a['data-product-sku'])

Raspando pagina 10
Raspando pagina 20
Raspando pagina 30
Raspando pagina 40
Raspando pagina 50
Raspando pagina 60
Raspando pagina 70
Raspando pagina 80


In [3]:
lista = []
lista_vinhos = []

for i in range(1,84):

    r = requests.get(f'https://www.wine.com.br/browse.ep?cID=100851&exibirEsgotados=false&pn={i}&listagem=horizontal&sorter=featuredProducts-desc&filters=cVINHOS')
    soup = BeautifulSoup(r.content,'html.parser')
    container = soup.find_all('div',{'class':'ProductDisplay-name'})
    for col in container: # contém todos os vinhos
        for art in col.find_all('a',href=True): #vinhos em cada coluna
            lista = 'https://www.wine.com.br{}'.format(art['href'])
            lista_vinhos.append(lista)

In [4]:
lista_vinhos[0:3]

['https://www.wine.com.br/vinhos/maison-le-star-rose-on-ice/prod18479.html',
 'https://www.wine.com.br/vinhos/champagne-montaudon-brut/prod3170.html',
 'https://www.wine.com.br/vinhos/ropiteau-pinot-noir-2017/prod22807.html']

# Extraindo Variáveis

In [5]:
price_Socio = []
price_nonSocio = []
for i, j in enumerate(lista_vinhos):
    prods_url = j
    resultado = requests.get(prods_url)
    bs = BeautifulSoup(resultado.content, 'html.parser')
    html_data_price = bs.find_all('div',{'class':'ProductSummary-club'})
    
    for elem in html_data_price:
        for dado in elem:
            if dado.string == 'Sócio:':
                data = elem.find('span',{'class':'Price-raw'}).string
                price_Socio.append({'sku':lst_sku[i],'price_Socio':data})
            elif dado.string == 'Não Sócio:':
                data = elem.find('span',{'class':'Price-raw'}).string
                price_nonSocio.append({'sku':lst_sku[i],'price_nonSocio':data})   

In [6]:
price_Socio = pd.DataFrame(price_Socio).set_index('sku')
price_Socio.head()

,price_Socio
sku,
18479,64.90
3170,276.90
22807,186.07
22698,554.90
21768,417.90


In [7]:
price_nonSocio = pd.DataFrame(price_nonSocio).set_index('sku')
price_nonSocio.head()

,price_nonSocio
sku,
18479,76.36
3170,325.77
22807,218.90
22698,652.83
21768,491.65


In [8]:
df = price_Socio.join([price_nonSocio], how = 'inner')
df =df.reset_index()

In [9]:
query = """ CREATE TABLE pricev1 ( sku TEXT,
                                 price_Socio TEXT,
                                 price_nonSocio TEXT) """



bd = Postgresql()
bd.create_table(query=query,user='postgres' , password='bruno22#' , host= 'localhost', port= '5432', database = 'brunods')
bd.insert_data(df = df, tabela = ' pricev1',user='postgres' , password='bruno22#' , host= 'localhost', port= '5432', database = 'brunods')

Tabela criada no postgresql com sucesso 
Conexão com Postgresql fechada
